# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

**Цели:**

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества F1 не меньше 0.75.

Инструкция по выполнению проекта

Загрузите и подготовьте данные.
Обучите разные модели.
Сделайте выводы.
Для выполнения проекта применять BERT необязательно, но вы можете попробовать.

# Описание данных

Данные находятся в файле toxic_comments.csv. Столбец text в нём содержит текст комментария, а toxic — целевой признак.

# Загрузка данных

https://drive.google.com/file/d/16M1xJA0tvDBT6T032BW6oYMFEspX8ETN/view?usp=sharing

In [ ]:
! gdown --id 16M1xJA0tvDBT6T032BW6oYMFEspX8ETN

/bin/bash: gdown: command not found


In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('wordnet') 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [ ]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [ ]:
data.duplicated().sum()

0

In [ ]:
data.isna().sum()

text     0
toxic    0
dtype: int64

**Вывод**

Загрузили данные, проверили и выяснили, что пропусков и дубликатов в данных нет.

# Лемматизация, токенизация

Токенизируем каждый твит, удалим стоп-слова, а также лемматизируем твиты.

In [ ]:
corpus = data['text']

In [ ]:
def tokenize(comment):
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    tokens = tokenizer.tokenize(comment)
    return tokens

In [ ]:
data['tokens'] = corpus.apply(lambda x: tokenize(x.lower()))

In [ ]:
stop_words = set(stopwords.words('english')) 

In [ ]:
def remove_stop_words(tokens):
    filtered_comment = [w for w in tokens if not w in stop_words]
    return filtered_comment

In [ ]:
data['wo_stopwords'] = data['tokens'].apply(lambda x: remove_stop_words(x))

In [ ]:
def lemmatize(tokens):
    wnl = WordNetLemmatizer()
    lemmas = [wnl.lemmatize(word) for word in tokens]
    return lemmas

In [ ]:
data['lemmas'] = data['wo_stopwords'].apply(lambda x: lemmatize(x))

In [ ]:
data.head()

,text,toxic,tokens,wo_stopwords,lemmas
0,Explanation\nWhy the edits made under my usern...,0,"[explanation, why, the, edits, made, under, my...","[explanation, edits, made, username, hardcore,...","[explanation, edits, made, username, hardcore,..."
1,D'aww! He matches this background colour I'm s...,0,"[d, aww, he, matches, this, background, colour...","[aww, matches, background, colour, seemingly, ...","[aww, match, background, colour, seemingly, st..."
2,"Hey man, I'm really not trying to edit war. It...",0,"[hey, man, i, m, really, not, trying, to, edit...","[hey, man, really, trying, edit, war, guy, con...","[hey, man, really, trying, edit, war, guy, con..."
3,"""\nMore\nI can't make any real suggestions on ...",0,"[more, i, can, t, make, any, real, suggestions...","[make, real, suggestions, improvement, wondere...","[make, real, suggestion, improvement, wondered..."
4,"You, sir, are my hero. Any chance you remember...",0,"[you, sir, are, my, hero, any, chance, you, re...","[sir, hero, chance, remember, page]","[sir, hero, chance, remember, page]"


**Вывод**

Разделили комментарии на токены, очистили от стоп-слов и привели к леммам.

# Обучение

## Logistic Regression

In [ ]:
train, test = train_test_split(data, test_size = 0.3, random_state = 42)

In [ ]:
count_tf_idf = TfidfVectorizer()

In [ ]:
target_train = train['toxic']
features_train = train['lemmas']
target_test = test['toxic']
features_test = test['lemmas']

In [ ]:
corpus_train = features_train.astype('U')
corpus_test = features_test.astype('U')

In [ ]:
tf_idf = count_tf_idf.fit_transform(corpus_train)

In [ ]:
lr = LogisticRegression(random_state=12345)
parameters = {'penalty':['l1', 'l2'],
              'C': [10,100],
              'solver':['liblinear'],
              'max_iter':[100,200],
              'class_weight':[None]}

clf = GridSearchCV(lr, param_grid = parameters, scoring = 'f1', cv = 5)

train_clf = clf.fit(tf_idf,target_train)
predictions_train = clf.predict(tf_idf)

f1 = f1_score(target_train, predictions_train)
print('f1 метрика на тренировочной выборке:', f1)

f1 метрика на тренировочной выборке: 0.9710183698947744


In [ ]:
clf.best_params_ 

{'C': 10,
 'class_weight': None,
 'max_iter': 100,
 'penalty': 'l1',
 'solver': 'liblinear'}

<div class="alert alert-block alert-info">
<b>Совет: </b> Напомню, что внутри кросс-валидации происходит разбиение выборки на треин и валидацию. Однако, в таком случае векторизатор обучен на всей выборке, а это не совсем корректно. Для избежания такого эффекта можно использовать <a href="https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html">пайплайн</a>. <a href="https://medium.com/analytics-vidhya/ml-pipelines-using-scikit-learn-and-gridsearchcv-fe605a7f9e05">Тут</a> есть пример.
</div>

## LGBMClassifier

In [ ]:
lgbm = LGBMClassifier(random_state = 42)

lgbm.fit(tf_idf,target_train)
predictions_train = lgbm.predict(tf_idf)

f1 = f1_score(target_train, predictions_train)
print('f1 метрика на тренировочной выборке:', f1)

f1 метрика на тренировочной выборке: 0.7884046136572419


**Вывод**

Обучили две модели: Logistic Regression и LightGBM Classifier. Лучшей оказалась Logistic Regression с метрикой f1 = 0,97

# Тестирование

## Logistic Regression

In [ ]:
tf_idf_test = count_tf_idf.transform(corpus_test)

In [ ]:
lr = LogisticRegression(random_state=12345, C = 10, class_weight = None, 
                        max_iter = 100, penalty = 'l1', solver = 'liblinear')
lr.fit(tf_idf,target_train)
predictions_test = lr.predict(tf_idf_test)
f1 = f1_score(target_test, predictions_test)

print('f1 метрика логистической регрессии на тестовой выборке:', f1)

f1 метрика логистической регрессии на тестовой выборке: 0.7788546255506609


## LGBMClassifier

In [ ]:
predictions_test = lgbm.predict(tf_idf_test)
f1 = f1_score(target_test, predictions_test)

print('f1 метрика LightGBM на тестовой выборке:', f1)

f1 метрика LightGBM на тестовой выборке: 0.757060794638583


**Вывод**

Протестировали модели: Logistic Regression и LightGBM Classifier. Лучшей оказалась Logistic Regression с метрикой f1 = 0,78

# Общий вывод

В данном проекте были обучены модели, чтобы классифицировать комментарии на позитивные и негативные. Перед этим комментарии были токенизированы, очищены от стоп-слов и лемматизированы.

Лучший результат f1 метрики показала модель логистической регрессии - 0.97 на тренировочной выборке и 0.78 на тестовой.